In [1]:
# Para testes
import numpy as np
import matplotlib.pyplot as plt

pi = np.pi

In [2]:
from typing import Callable

# Cinemática Inversa

O problema da cinemática inversa consiste em encontrar ângulos de juntas robóticas que correspondem à certa posição do braço robótico no espaço. Ou seja, encontrar valores de $\theta$ que satisfaçam $f(\theta)=x$, $f(\theta_1,\theta_2)=(x,y)$ e $f(\theta_1,\theta_2,\theta_3)=(x,y,z)$.

## Método de Newton

In [3]:
def nr(f: Callable[[float], float], df: Callable[[float], float], x: float, p: float) -> float:
    # p = chute inicial
    def g(w):
        return f(w) - x
    xl = p
    # critérios de parada
    N, e_stop = 50, 1e-8
    iter, e = 0, 1
    while iter < N-1 and e > e_stop:
        xn = xl - g(xl)/df(xl)
        e = xn
        xl = xn
        iter += 1

    return xn

## Caso para 1 dimensão

O caso para 1 dimensão se resume em encontrar um ângulo referente a uma posição em um círculo

In [4]:
R = 3

def f_1d(theta: float) -> float:
    return theta*R

def d_f_1d(theta: float) -> float:
    return R

In [5]:
theta_nr = nr(f_1d, d_f_1d, 2, 0.7)
print(f"Valor de theta encontrado para x = 2: {theta_nr}")
print(f"Valor de x: {f_1d(theta_nr)}")

Valor de theta encontrado para x = 2: 0.6666666666666667
Valor de x: 2.0


## Caso para 2 dimensões

Nesse caso se tem um um sistema com dois braços, braço 1 e braço 2. A posição desejada se dará pela posição da ponta não articulada do braço 2. Considerando os ângulos $\theta_1$ (calculado entre a reta do braço 1 e a reta $x=0$) e $\theta_2$ (calculado entre a reta do braço 2 e a paralela à reta $x=0$), a posição desejada se dá por:

$(x,y)=\Big(f_1(\theta_1,\theta_2), f_2(\theta_1,\theta_2)\Big)=\Big(R_1\cos(\theta_1)+R_2\cos(\theta_2), R_1\sin(\theta_1)+R_2\sin(\theta_2)\Big)$

$\frac{df}{d\theta}=
\begin{bmatrix}
\frac{\partial f_1}{\partial\theta_1}&\frac{\partial f_1}{\partial\theta_2}\\
\frac{\partial f_2}{\partial\theta_1}&\frac{\partial f_2}{\partial\theta_2}
\end{bmatrix}=
\begin{bmatrix}
-R_1\sin(\theta_1)&-R_2\sin(\theta_2)\\
R_1\cos(\theta_1)&R_2\cos(\theta_2)
\end{bmatrix}$

### Método da descida do gradiente

Nesse caso, deve-se aplicar o método para $f_1$ e $f_2$.
As funções a serem minimizadas serão
$\Big(f_1(\theta_1, \theta_2) - x\Big)^2$ e $\Big(f_2(\theta_1, \theta_2) - y\Big)^2$.

$\nabla f_1 =
\Big( 2\cdot\big[f_1(\theta_1, \theta_2) - x\big]\frac{\partial f_1}{\partial\theta_1},
2\cdot\big[f_1(\theta_1, \theta_2) - x\big]\frac{\partial f_1}{\partial\theta_2} \Big)$

$\nabla f_2 =
\Big( 2\cdot\big[f_2(\theta_1, \theta_2) - y\big]\frac{\partial f_2}{\partial\theta_1},
2\cdot\big[f_2(\theta_1, \theta_2) - y\big]\frac{\partial f_2}{\partial\theta_2} \Big)$

**OBS:** Assim não funcionaria, pois seria possível encontrar valores diferentes para $\theta_1$ e $\theta_2$ em $f_1$ e $f_2$.

In [6]:
R1, R2 = 3, 2.5

# f_2d: (R1*np.cos(theta1)+R2*np.cos(theta2), R1*np.sin(theta1)+R2*np.sin(theta2))
# grad_f_2d: ( (-R1*np.sin(theta1), -R2*np.sin(theta2)), (R1*np.cos(theta1), R2*np.cos(theta2)) )

def f(theta1: float, theta2: float, func: Callable) -> float:
    return R1*func(theta1) + R2*func(theta2)

def f_2d_x(theta1: float, theta2: float, x: float) -> float:
    return (f(theta1, theta2, np.cos) - x)**2

def f_2d_y(theta1: float, theta2: float, y: float) -> float:
    return (f(theta1, theta2, np.sin) - y)**2

def grad_f_2d_x(theta1: float, theta2: float, x: float) -> tuple:
    w = (f(theta1, theta2, np.cos)-x)
    return ((2*w*(-R1*np.sin(theta1))),
            (2*w*(-R2*np.sin(theta2))))

def grad_f_2d_y(theta1: float, theta2: float, y: float) -> tuple:
    w = (f(theta1, theta2, np.sin)-y)
    return (2*w*R1*np.cos(theta1),
            2*w*R2*np.cos(theta2))

In [7]:
def descida_grad(f: Callable[[float, float, float], float],
                 df: Callable[[float, float, float], tuple],
                 k: float, x0y0: tuple = (0.5, 0.5)) -> tuple:
    # passo fixo
    h = 0.5
    # x0y0 = chute inicial
    xo, yo = x0y0[0], x0y0[1]
    # critérios de parada
    N, tol = 50, 1e-8
    iter, diff = 0, tol + 1
    while iter < N and diff > tol:
        grad0, grad1 = df(xo, yo, k)
        diff = f(grad0, grad1, k)
        xnyn = (
            (xo - grad0*h),
            (yo - grad1*h)
        )
        xo, yo = xnyn[0], xnyn[1]
        iter += 1
    print(f"iter = {iter}")
    return xnyn

In [8]:
# x
x = 2
theta1, theta2 = descida_grad(f_2d_x, grad_f_2d_x, x, (0.8, 0.8))
print(f"Valores de theta encontrados para x = {x}: {theta1, theta2}")
print(f"Valor de x: {f(theta1, theta2, np.cos)}")
print(f"deveria ser 0 mas é: {f_2d_x(theta1, theta2, 2)}")

iter = 50
Valores de theta encontrados para x = 2: (-6.034381671028512, 44.52021439823019)
Valor de x: 5.054567235343995
deveria ser 0 mas é: 9.330380995237059


### Método da bisseção

In [9]:
def norm(v: tuple) -> float:
    return np.sqrt(sum([i**2 for i in v]))

def dist(v1: tuple, v2: tuple) -> tuple:
    n = len(v1)
    diff = []
    if n != len(v2):
        return False
    for i in range(n):
        diff.append(v1[i] - v2[0])
    return tuple(diff)

In [10]:
def bis(f: Callable[[float, float, float, Callable], float],
        func: Callable, a0: tuple, b0: tuple, k: float) -> tuple:
    xaya, xbyb, xmym = a0, b0, [None, None]
    N, e_stop = 150, 1e-16
    iter, e = 0, abs(norm(dist(b0, a0)))
    loop = True
    while loop:
        xmym = list(xmym)
        xmym[0] = (xaya[0] + xbyb[0])/2
        xmym[1] = (xaya[1] + xbyb[1])/2
        xmym = tuple(xmym)
        test = f(xaya[0], xaya[1], k, func) * f(xmym[0], xmym[1], k, func)
        #print(f"test={test}")
        if test > 0:
            xaya = xmym
        elif test < 0:
            xbyb = xmym
        else:
            loop = False
        iter += 1
        e = abs(norm(dist(xbyb, xaya)))
        loop = iter < N and e > e_stop

    return xmym

In [11]:
def f_k(theta1: float, theta2: float, k: float, func: Callable) -> float:
    return R1*func(theta1) + R2*func(theta2) - k

In [12]:
# x
x = 2
a0 = (np.pi/4, np.pi/4)
b0 = (5*np.pi/4, 5*np.pi/4)
#print(f"f_k(a0)={f_k(a0[0], a0[1], x, np.cos)}")
#print(f"f_k(b0)={f_k(b0[0], b0[1], x, np.cos)}")
theta1, theta2 = bis(f_k, np.cos, a0, b0, x)
print(f"Valores de theta encontrados para x = {x}: {theta1, theta2}")
print(f"Valor de x: {f(theta1, theta2, np.cos)}")

Valores de theta encontrados para x = 2: (1.1986277928345705, 1.1986277928345705)
Valor de x: 2.0000000000000004


# Cinemática inversa numérica

Sendo $x=f(\theta)$ o vetor das posições referentes aos ângulos ($\theta\in\mathbb{R}^n$), e $x_d \in\mathbb{R}^m$ as coordenadas desejadas, a solução será $\theta_d$ tal que $g(\theta_d)=x_d-f(\theta_d)=0$.

Com isso, usando expansão de Taylor, tomando um valor inicial $\theta_0$:

$x_d=f(\theta_d)=f(\theta_0)+\underset{J(\theta_0)}{\boxed{\frac{\partial f}{\partial\theta}\Big|_{\theta_0}}}
\cdot\underset{\Delta\theta}{\boxed{(\theta_d-\theta_0)}}~,~~J(\theta_0)\in\mathbb{R}^{m\times n}~\Rightarrow~$
$J(\theta_0)\Delta\theta=x_d-f(\theta_0)~\Rightarrow$

$\theta_1=\theta_0+\Delta\theta$

Generalizando:

$\theta_{k+1}=\theta_k+\Delta\theta_k$

**Obs:** Sendo $f(\theta) = \big[f_1(\theta)~,~~f_2(\theta)~,~~\cdots~,~~f_n(\theta)\big]$:

$J(\theta)=$
$\begin{bmatrix}
\frac{\partial f_1}{\partial\theta_1}(\theta)&\cdots&\frac{\partial f_1}{\partial\theta_n}(\theta)\\
\vdots&\ddots&\vdots\\
\frac{\partial f_n}{\partial\theta_1}(\theta)&\cdots&\frac{\partial f_n}{\partial\theta_n}(\theta)
\end{bmatrix}$

## Caso para 2 dimensões

$f=(x,y)=\Big(f_1(\theta_1,\theta_2),~f_2(\theta_1,\theta_2)\Big)=\Big(R_1\cos(\theta_1)+R_2\cos(\theta_2),~R_1\sin(\theta_1)+R_2\sin(\theta_2)\Big)$

$J=
\begin{bmatrix}
\frac{\partial f_1}{\partial\theta_1}&\frac{\partial f_1}{\partial\theta_2}\\
\frac{\partial f_2}{\partial\theta_1}&\frac{\partial f_2}{\partial\theta_2}
\end{bmatrix}=
\begin{bmatrix}
-R_1\sin(\theta_1)&-R_2\sin(\theta_2)\\
R_1\cos(\theta_1)&R_2\cos(\theta_2)
\end{bmatrix}$

**Taylor:**

$\begin{bmatrix}
-R_1\sin(\theta_{1_k})&-R_2\sin(\theta_{2_k})\\
R_1\cos(\theta_{1_k})&R_2\cos(\theta_{2_k})
\end{bmatrix}$
$\begin{bmatrix}
\theta_{1_{k+1}}-\theta_{1_k}\\
\theta_{2_{k+1}}-\theta_{2_k}\\
\end{bmatrix}=$
$\begin{bmatrix}
x_{1_d}\\x_{2_d}
\end{bmatrix}-$
$\begin{bmatrix}
R_1\cos(\theta_{1_k})+R_2\cos(\theta_{2_k})\\
R_1\sin(\theta_{1_k})+R_2\sin(\theta_{2_k})
\end{bmatrix}$

$
\begin{bmatrix}
\theta_{1_{k+1}}-\theta_{1_k}\\
\theta_{2_{k+1}}-\theta_{2_k}
\end{bmatrix}=
\begin{bmatrix}
-R_1\sin(\theta_{1_k})&-R_2\sin(\theta_{2_k})\\
R_1\cos(\theta_{1_k})&R_2\cos(\theta_{2_k})
\end{bmatrix}^{-1}
\begin{bmatrix}
x_{1_d} - \big(R_1\cos(\theta_{1_k})+R_2\cos(\theta_{2_k})\big)\\
x_{2_d} - \big(R_1\sin(\theta_{1_k})+R_2\sin(\theta_{2_k})\big)
\end{bmatrix}
$

$
\Theta = A^{-1}\cdot\Big[X-B\Big]
$

In [17]:
def A_matrix_maker(R1, R2, theta1, theta2):
    return [[-R1*np.sin(theta1), -R2*np.sin(theta2)], [R1*np.cos(theta1), R2*np.cos(theta2)]]
def B_matrix_maker(R1, R2, theta1, theta2):
    return [R1*np.cos(theta1)+R2*np.cos(theta2), R1*np.sin(theta1)+R2*np.sin(theta2)]

In [22]:
theta1_0, theta2_0 = (0, np.pi/4)
#theta2_0 = (5*np.pi/4, 5*np.pi/4)

xd = np.asarray([4, 4])

for i in range(50):
    A_matrix = np.asarray(A_matrix_maker(R1, R2, theta1_0, theta2_0))
    B_matrix = np.asarray(B_matrix_maker(R1, R2, theta1_0, theta2_0))
    Theta_vector = np.matmul(np.linalg.inv(A_matrix), xd - B_matrix)
    theta1_0, theta2_0 = Theta_vector[0] + theta1_0, Theta_vector[1] + theta2_0
    
print(f"Valores de theta encontrados para x,y = {xd}: {theta1_0, theta2_0}")
print(f"Valor de (x,y): {B_matrix_maker(R1, R2, theta1_0, theta2_0)}")

Valores de theta encontrados para x,y = [4 4]: (-55.519271996805465, 69.6070366266278)
Valor de (x,y): [3.7494869522765764, 3.7519334551698993]
